In [4]:
import CaboCha
import pandas as pd


In [5]:
from pandas import DataFrame
df: DataFrame = pd.read_table('./data/ai.ja/ai.ja.txt', header=None, names=['sentence'])

In [6]:
from CaboCha import Tree
from typing import List

def printTree(tree: Tree):
    print(tree.toString(CaboCha.FORMAT_LATTICE))

def headTrees(trees: List[Tree], from_int: int, to_int: int) -> None:
    for tree in trees[from_int:to_int]:
        printTree(tree)



In [7]:
# parse()の返り値が最新のTreeオブジェクトになっている問題に対する試み(1)
# toString() を呼び出せばポイント参照から実態の参照へと切り替わるのでは？と期待したのだが、失敗。
# def noLazyParse(sentence: str) -> Tree:
#     parsed = c.parse(sentence)
#     parsed.toString(CaboCha.FORMAT_LATTICE)
#     return parsed

# trees = df['sentence'].map(noLazyParse)
# trees.head()

In [8]:
# parse()の返り値が最新のTreeオブジェクトになっている問題に対する試み(2)
# いっそ要素ごとにParserを作ってしまおう、とやってみたが、iPhythonのKernelが落ちる。
# trees = df['sentence'].map(CaboCha.Parser().parse)
# trees.head()

In [16]:
# parse()の返り値が最新のTreeオブジェクトになっている問題に対する試み(3)
# parse()実行直後にプロパティをコピーするしかない。

from CaboCha import Tree
class TreeObject:
    def __init__(self, tree: Tree):
        self._str = tree.toString(CaboCha.FORMAT_LATTICE)
        self.sentence = tree.sentence()
        self.chunks = [tree.chunk(i) for i in range(tree.chunk_size())]
        self.tokens = [tree.token(i) for i in range(tree.token_size())]

    def __str__(self) -> str:
        return self._str

    def __repr__(self) -> str:
        return self._str

c = CaboCha.Parser()

trees = df['sentence'].map(lambda s: TreeObject(c.parse(s)))
trees.head()

0    * 0 -1D 1/1 0.000000\n人工\t名詞,一般,*,*,*,*,人工,ジンコ...
1    * 0 17D 1/1 0.388993\n人工\t名詞,一般,*,*,*,*,人工,ジンコ...
2    * 0 1D 6/7 3.194287\n『\t記号,括弧開,*,*,*,*,『,『,『\n...
3    * 0 1D 0/1 1.813378\n人間\t名詞,一般,*,*,*,*,人間,ニンゲン...
4    * 0 1D 1/2 0.906609\nプログラミング\t名詞,サ変接続,*,*,*,*,...
Name: sentence, dtype: object